In [35]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import src.data.breathe_data as bd
import src.data.helpers as dh
import src.inference.helpers as ih
import src.modelling_ar.ar as ar
import src.inf_cutset_conditioning.helpers as cutseth
import src.inf_cutset_conditioning.cutset_cond_algs_learn_ar_change as cca_ar_change
import src.models.helpers as mh

In [2]:
# Load data
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_conservative_smoothing_with_idx")

INFO:root:* Checking for same day measurements *


In [30]:
id = "527"
dftmp, start_idx, end_idx = dh.find_longest_consec_series(df[df.ID == id], n_days=1)


ecfev1_noise_model_suffix = "_std_0.068"
ar_prior = "breathe (2 days model, ecFEV1 addmultnoise, ecFEF25-75)"
ar_change_cpt_suffix = "_shape_factor"

(
    p_M_given_D,
    log_p_D_given_M,
    AR_given_M_and_D,
    AR_given_M_and_all_D,
    res_dict,
) = cca_ar_change.run_long_noise_model_through_time(
    dftmp,
    ar_prior=ar_prior,
    ar_change_cpt_suffix=ar_change_cpt_suffix,
    ecfev1_noise_model_suffix=ecfev1_noise_model_suffix,
)

# 5.61 for 4 entries without
# 5.40 when removing AR calculations

KeyboardInterrupt: 

In [31]:
p_S_given_M = [
    [
        177,
        np.array(
            [
                9.30909107e-01,
                6.11579723e-02,
                6.64331139e-03,
                1.05999387e-03,
                2.29614946e-04,
            ]
        ),
    ],
    [
        180,
        np.array(
            [
                9.89090023e-01,
                1.05663301e-02,
                3.21210807e-04,
                2.02459169e-05,
                2.18995184e-06,
            ]
        ),
    ],
    [
        139,
        np.array(
            [
                9.79577461e-01,
                1.95862871e-02,
                7.73523038e-04,
                5.60533868e-05,
                6.67542188e-06,
            ]
        ),
    ],
    [
        272,
        np.array(
            [
                9.39907334e-01,
                5.34324886e-02,
                5.65438269e-03,
                8.41964226e-04,
                1.63830104e-04,
            ]
        ),
    ],
    [
        146,
        np.array(
            [
                9.99891829e-01,
                1.08103981e-04,
                6.68738524e-08,
                1.79743333e-10,
                1.51356715e-12,
            ]
        ),
    ],
    [
        117,
        np.array(
            [
                9.99988755e-01,
                1.12438485e-05,
                1.54107847e-09,
                1.27624351e-12,
                4.11774785e-15,
            ]
        ),
    ],
    [
        201,
        np.array(
            [
                9.87612956e-25,
                7.92815358e-06,
                7.37819775e-02,
                4.96378968e-01,
                4.29831127e-01,
            ]
        ),
    ],
]

In [22]:
len(p_S_given_M)

7

In [45]:
p_S_given_M
S = mh.DiscreteVariableNode("AR change factor shape", 2, 10, 2)

fig = make_subplots(rows=len(p_S_given_M) + 1, cols=1, shared_xaxes=True)
# fig = make_subplots(1, cols=1, shared_xaxes=True)

p_S_given_all_M = np.ones(5)

for i, (idx, p_S) in enumerate(p_S_given_M):
    p_S_given_all_M *= p_S
    p_S_given_all_M = p_S_given_all_M / np.sum(p_S_given_all_M)
    fig.add_trace(
        # Add histogram
        go.Histogram(
            x=p_S,
            name=f"{idx} - p(S|D)",
            xbins=dict(
                
            )
        ),
        row=i + 1,
        col=1
    )

fig.add_trace(
    go.Bar(x=S.values, y=p_S_given_all_M, name="p(S|all M)"),
    row=1,
    col=1,
)
fig.show()